In [69]:
TEST_MODE = True

In [1]:
execfile('shell_init.py')

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.4.1
      /_/

Using Python version 2.7.11 (default, Feb 15 2016 00:04:52)
SparkContext available as sc, HiveContext available as sqlContext.


In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [32]:
raw = sc.textFile("/user/smartanalytics/customer_data/01_personal_data.csv")

In [61]:
header = (raw.zipWithIndex()
            .filter(lambda (x,y): y==0)
            .collect()[0])
header = header[0].split(',')

In [71]:
field_names = ['', 'state', 'name']
field_names2 = ['id', 'state', 'name']
selected_ids = [h in field_names for h in header]

In [66]:
def drop_columns(s):
    s = s.split(',')
    s2 = [si for si,val in zip(s,selected_ids) if val]
    return s2

customers = (raw.zipWithIndex()
            .filter(lambda (x,y): y>0)
            .map(lambda (x,y): x)
            .map(drop_columns))

In [73]:
if TEST_MODE:
    field_names2 = field_names2+['twitter_id']
    customers = customers.map(lambda x:x+[''])

In [77]:
customers_df = customers.toDF(field_names2)
url = "jdbc:postgresql://localhost/customer?user=postgres"
customers_df.write.jdbc(url=url, table="customers", mode="overwrite")